In [ ]:
!pip install langchain_community groq langgraph python-dotenv

In [19]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 71.6 MB/s eta 0:00:00


In [11]:
import os
import ast
import getpass
from typing import List, TypedDict, Literal
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from groq import Groq
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv

In [12]:
# --- API Key Management ---
def get_api_key():
    """Securely gets the Groq API key from environment variables or user input."""
    if "GROQ_API_KEY" in os.environ:
        print("Found GROQ_API_KEY in environment variables.")
        return os.environ["GROQ_API_KEY"]
    else:
        print("GROQ_API_KEY not found in environment variables.")
        # Use getpass for secure, non-echoed input
        return getpass.getpass("Please enter your Groq API key: ")

api_key = get_api_key()
if not api_key:
    print("No API key provided. Exiting.")
    exit()

# Initialize the Groq client with the obtained key
client = Groq(api_key=api_key)

GROQ_API_KEY not found in environment variables.
Please enter your Groq API key: ··········


In [13]:
# --- State Definition ---
class AgentState(TypedDict):
    question: str
    documents: List[str]
    generation: str
    is_code: bool
    code_type: Literal["python", "cpp", "none"]
    validation_error: str
    retries: int
    final_answer: str

In [21]:
# --- Graph Nodes ---
def retrieve_documents(state: AgentState):
    """
    Node to retrieve relevant documents from the vector store.
    """
    print("\n--- 📄 Retrieving Documents ---")
    question = state["question"]
    documents = retriever.invoke(question)
    document_contents = [doc.page_content for doc in documents]
    print(f"Retrieved {len(document_contents)} documents.")
    return {"documents": document_contents}

def generate_code_or_doc(state: AgentState):
    """
    Node to generate a response using the Groq LLM.
    """
    print("--- 🤖 Generating Response ---")
    question = state["question"]
    documents = state["documents"]
    validation_error = state.get("validation_error", "")
    retries = state.get("retries", 0)

    system_prompt = (
        "You are an expert programmer for embedded systems (Arduino, Raspberry Pi) and a technical writer. "
        "Provide a complete, runnable response. For Arduino, use C++ (.ino format). For Raspberry Pi, use Python with RPi.GPIO."
    )
    user_prompt_parts = ["Context:\n" + "\n".join(documents)]

    if validation_error:
        print(f"--- ⚠️ Retrying (Attempt {retries}) with validation error ---")
        user_prompt_parts.append(f"The previous attempt failed: '{validation_error}'. Please provide a corrected script.")

    user_prompt_parts.append(f"User's Question: {question}")
    user_prompt = "\n\n".join(user_prompt_parts)

    completion = client.chat.completions.create(
        model="openai/gpt-oss-20b",
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        temperature=0.7, max_tokens=8192, top_p=1, stream=False, stop=None,
        tools=[{"type":"browser_search"},{"type":"code_interpreter"}]
    )
    generation = completion.choices[0].message.content

    python_keywords = ["def ", "import RPi.GPIO", "class "]
    cpp_keywords = ["void setup()", "void loop()", "#include", "pinMode"]
    is_python = any(keyword in generation for keyword in python_keywords)
    is_cpp = any(keyword in generation for keyword in cpp_keywords)

    code_type = "python" if is_python else "cpp" if is_cpp else "none"
    is_code = is_python or is_cpp

    print(f"Generation complete. Identified as code: {is_code}. Type: {code_type}.")
    return {"generation": generation, "is_code": is_code, "code_type": code_type, "retries": retries + 1}

def clean_code(code_string: str, language: str) -> str:
    """Helper to remove markdown fences from code blocks."""
    return code_string.replace(f"```{language}", "").replace("```", "").strip()

def validate_python_code(state: AgentState):
    """
    Node to validate generated Python code for syntax errors.
    """
    print("--- 🐍 Validating Python Code ---")
    code_to_validate = clean_code(state["generation"], "python")
    try:
        ast.parse(code_to_validate)
        print("Validation successful: Python code is syntactically correct.")
        return {"validation_error": None}
    except SyntaxError as e:
        error_message = f"Syntax Error: {e}"
        print(f"Validation failed: {error_message}")
        return {"validation_error": error_message}

def validate_cpp_code(state: AgentState):
    """
    Node for basic syntax validation of Arduino (C++) code.
    """
    print("--- 🔬 Validating C++/Arduino Code ---")
    code_to_validate = clean_code(state["generation"], "cpp")
    errors = []
    if code_to_validate.count('{') != code_to_validate.count('}'):
        errors.append("Mismatched curly braces {}.")
    if not errors:
        print("Validation successful: C++/Arduino code seems syntactically plausible.")
        return {"validation_error": None}
    else:
        error_message = " | ".join(errors)
        print(f"Validation failed: {error_message}")
        return {"validation_error": error_message}

def prepare_final_output(state: AgentState):
    """
    Node to prepare the final answer.
    """
    print("--- ✅ Preparing Final Output ---")
    return {"final_answer": state["generation"]}

# --- Conditional Edges for Routing ---

def route_to_validation(state: AgentState):
    """Router to decide whether to validate code, retry, or finish."""
    if state["is_code"]:
        return state["code_type"]
    return "end"

def route_after_validation(state: AgentState):
    """Router to decide whether to retry generation or finish."""
    if state["validation_error"]:
        if state["retries"] < MAX_RETRIES:
            return "retry"
        else:
            print("--- ❌ Max retries reached. Finishing. ---")
            return "end"
    return "end"

# --- Main Execution Block ---


In [20]:
MAX_RETRIES = 2

print("\n--- Setting up RAG System ---")
print("Initializing embedding model (this may take a moment on first run)...")
# Using a smaller, efficient open-source embedding model
embedding_model = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-base")

docs = [
    "To blink an LED with Arduino, use pinMode() in setup() and digitalWrite() in loop().",
    "Raspberry Pi uses the RPi.GPIO Python library. Use GPIO.setup() and GPIO.output(). Crucially, always run GPIO.cleanup().",
    "A pull-up resistor ensures a known state for a digital input. Arduino has internal pull-ups enabled via pinMode(pin, INPUT_PULLUP)."
]
vectorstore = FAISS.from_texts(texts=docs, embedding=embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
print("RAG system is ready.")

# --- Build the Graph ---
workflow = StateGraph(AgentState)
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("generate", generate_code_or_doc)
workflow.add_node("validate_python", validate_python_code)
workflow.add_node("validate_cpp", validate_cpp_code)
workflow.add_node("prepare_output", prepare_final_output)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_conditional_edges(
    "generate",
    route_to_validation,
    {"python": "validate_python", "cpp": "validate_cpp", "end": "prepare_output"}
)
workflow.add_conditional_edges(
    "validate_python",
    route_after_validation,
    {"retry": "generate", "end": "prepare_output"}
)
workflow.add_conditional_edges(
    "validate_cpp",
    route_after_validation,
    {"retry": "generate", "end": "prepare_output"}
)
workflow.add_edge("prepare_output", END)
app = workflow.compile()

# --- Run Example Queries ---
queries = [
    "Give me a complete Arduino sketch to blink the built-in LED every second.",
    "Write a Python script for Raspberry Pi to detect a button press on GPIO 17."
]
for i, q in enumerate(queries):
    print(f"\n{'='*50}\n🚀 Executing Query {i+1}: '{q}'\n{'='*50}")
    inputs = {"question": q, "retries": 0}
    result = app.invoke(inputs)

    print("\n\n--- ✨ Final Answer ✨ ---")
    print(result.get('final_answer', 'No final answer was generated.'))
    print("--------------------------\n")




--- Setting up RAG System ---
Initializing embedding model (this may take a moment on first run)...


RAG system is ready.

🚀 Executing Query 1: 'Give me a complete Arduino sketch to blink the built-in LED every second.'

--- 📄 Retrieving Documents ---
Retrieved 2 documents.
--- 🤖 Generating Response ---


Generation complete. Identified as code: True. Type: cpp.
--- 🔬 Validating C++/Arduino Code ---
Validation successful: C++/Arduino code seems syntactically plausible.
--- ✅ Preparing Final Output ---


--- ✨ Final Answer ✨ ---
Below is a minimal, ready‑to‑upload Arduino sketch that blinks the built‑in LED (normally pin 13 on most boards) once per second.

```cpp
/*  BlinkBuiltIn.ino
 *  Blink the Arduino's built‑in LED every 1 second.
 *  Works on all boards that expose LED_BUILTIN (e.g. Uno, Nano, Mega, Leonardo).
 */

void setup() {
  // Set the built‑in LED pin as an output.
  pinMode(LED_BUILTIN, OUTPUT);
}

void loop() {
  // Turn the LED on
  digitalWrite(LED_BUILTIN, HIGH);
  delay(500);          // wait 0.5 s

  // Turn the LED off
  digitalWrite(LED_BUILTIN, LOW);
  delay(500);          // wait 0.5 s
}
```

**How to use**

1. Open the Arduino IDE (or your preferred editor).
2. Create a new sketch and paste the code above.
3. Select the correct board and COM port under **Tools*